## Format a series of wavefiles for use with the open_ephys system and the boner stimulus rig

- See function make_stereo_stim.


In [ ]:
import socket
import os
import logging

from swissknife.bci.core import expstruct as et
from helperephys.boner import stimprep

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.debug('all modules loaded')

comp_name=socket.gethostname()
logger.info('Computer: ' + comp_name)

### Get a list of all of the wav files in a folder

In [ ]:
raw_stim_folder = os.path.join('/mnt/cube/earneodo', 'bci_zf', 'stim_data', 'b1159','001')
f_list = [f for f in os.listdir(raw_stim_folder) if f.split('.')[-1] == 'wav']
print(f_list)

### Define sampling rate for final files and frequency for the tags
If the frequency dictionary is skipped, the function create_bb_stim will do 1Khz for all stimuli

In [ ]:
### Manually enter the tags
# otherwise they will all be 1000Hz
sampling_rate = 30000 # sampling rate of the output files

stims_to_tag = {'bos01': 1000,
                'bos02': 2000,
                'bos03': 3000,
                'bos03-rev': 4000,
                'bos05': 5000}

stimprep.create_bb_stim(f_list, raw_stim_folder, sampling_rate, stim_tag_dict=stims_to_tag)

2018-04-26 19:32:51,823 helperephys.boner.stimprep INFO     Processing /mnt/cube/earneodo/bci_zf/stim_data/b1159/001/bos02.wav
2018-04-26 19:32:51,823 helperephys.boner.stimprep INFO     Processing /mnt/cube/earneodo/bci_zf/stim_data/b1159/001/bos02.wav
2018-04-26 19:32:51,825 helperephys.boner.stimprep INFO     tag_freq = 2000
2018-04-26 19:32:51,825 helperephys.boner.stimprep INFO     tag_freq = 2000
2018-04-26 19:32:51,832 helperephys.boner.stimprep INFO     Will resample from 1751201 to 1191293 sampes
2018-04-26 19:32:51,832 helperephys.boner.stimprep INFO     Will resample from 1751201 to 1191293 sampes


In [ ]:
def create_bb_stim(file_list, location_fold, out_s_f, stim_tag_dict=None):
    out_dir = os.path.join(location_fold, 'bb_stim')
    if not os.path.exists(location_fold):
        os.makedirs(directory)
    for stim_f_name in file_list:
        stim_name = stim_f_name.split('.')[0]
        wave_in = os.path.join(location_fold, '{}.wav'.format(stim_name))
        print('Processing {}'.format(wave_in))
        if stim_tag_dict:
            tag_freq = stim_tag_dict[stim_name]
            print('tag_freq = {}'.format(tag_freq))
            song_out = stimprep.make_stereo_stim(wave_in, out_s_f, tag_freq=tag_freq)
        else:
            song_out = stimprep.make_stereo_stim(wave_in, out_s_f)
        wave_out = os.path.join(out_dir, '{}_tag.wav'.format(stim_name))
        wavfile.write(wave_out, out_sf, song_out)
        print(wave_out)
        
        tags_par_file = os.path.join(out_dir, 'stim_tags.yml')
        with open(tags_par_file, 'w') as outfile:
            yaml.dump(stims_to_tag, outfile, default_flow_style=False)

create_bb_stim(f_list, raw_stim_folder, sampling_rate, stim_tag_dict=stims_to_tag)

In [ ]:
for stim_name, tag_f in stims_to_tag.items():
    wave_in = os.path.join(fn['folders']['stim'], '{}.wav'.format(stim_name))
    print(wave_in)
    wave_out = os.path.join(fn['folders']['stim'], '{}_tag.wav'.format(stim_name))
    song_out = make_stereo_stim(wave_in, tag_f, out_sf)
    wavfile.write(wave_out, out_sf, song_out)
    print(wave_out)
    
tags_par_file = os.path.join(fn['folders']['stim'], 'stim_tags.yml')
with open(tags_par_file, 'w') as outfile:
    yaml.dump(stims_to_tag, outfile, default_flow_style=False)

In [ ]:
# THIS IS THE ACTUAL THING
def make_stereo_stim(wave_in, tag_freq, out_sf):
    in_sf, data = wavfile.read(wave_in)
    new_len = int(round(data.shape[0] * out_sf / in_sf))
    new_len = data.shape[0]
    #song_out = np.round(resample(data, new_len)).astype(np.int16)
    song_out = data
    tag_out = np.round(np.sin(2*np.pi*tag_freq*np.arange(new_len)/out_sf)*30000).astype(np.int16)
    
    return np.column_stack([song_out, tag_out])

bird = 'b1067'
stim_sess = '001'
out_sf = 30000

# stims_to_tag = {'bos': 1000,
#                 'con': 3000,
#                 'bos_rev': 5000}

# stims_to_tag = {'bos': 1000,
#                 'con': 2000,
#                 'bos_rev': 3000,
#                 'bos_syn': 4000}

stims_to_tag = {'bos01': 1000,
                'bos05-rev': 7000,
                'bos02': 2000,
                'bos03': 3000,
                'bos04': 4000,
                'bos05': 5000}

# stims_to_tag = {'bos05': 6000,
#                 'bos06': 500}

# stims_to_tag = {'bos01': 1000,
#                 'bos01_rev': 5000,
#                 'con01': 4000,
#                 'bos01_syn': 2000}

# stims_to_tag = {'bos01': 1000,
#                 'bos02': 2000,
#                 'bos03': 3000,
#                 'con01': 4000}


#stims_to_tag = {'bos01_rev': 500}
#stims_to_tag = {'bos03_rev': 1500}

#wave_out = os.path.join(os.path.abspath('/mnt/cube/earneodo/bci_zf/stim_data/z021/001'), 'bos_syn_tag.wav')
fn = et.file_names(bird, stim_sess)
for stim_name, tag_f in stims_to_tag.items():
    wave_in = os.path.join(fn['folders']['stim'], '{}.wav'.format(stim_name))
    print(wave_in)
    wave_out = os.path.join(fn['folders']['stim'], '{}_tag.wav'.format(stim_name))
    song_out = make_stereo_stim(wave_in, tag_f, out_sf)
    wavfile.write(wave_out, out_sf, song_out)
    print(wave_out)
    
tags_par_file = os.path.join(fn['folders']['stim'], 'stim_tags.yml')
with open(tags_par_file, 'w') as outfile:
    yaml.dump(stims_to_tag, outfile, default_flow_style=False)

In [ ]:
tags_par_file = os.path.join(fn['folders']['stim'], 'stim_tags.yaml')
with open(tags_par_file, 'w') as outfile:
    yaml.dump(stims_to_tag, outfile, default_flow_style=False)

In [ ]:
bird = 'z023'
et.file_names(bird, '001')

In [ ]:
plt.plot(song_out.T[:1000])

In [ ]:
song_out.shape

In [ ]:
A1x16-5mm-50
passaro16-nn_acute